In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..','..')))
from edbo.plus.optimizer_botorch import EDBOplus

c:\Users\D.Dalmau\anaconda3\envs\edbo+\lib\site-packages\torch\torch_version.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import packaging  # type: ignore[attr-defined]


#### This is a tutorial that covers the basics for running EDBO+: from designing a combinatorial space to running the Bayesian Optimizer.

## 1. Creating a search scope using EDBO+.

##### To run EDBO+ we need to first create a reaction scope (search space) in a .csv format with all the possible combinations that you want to consider for our optimization. 
##### You can "manually" create a reaction scope using any spreadsheet editor (such as Excel, Libreoffice Calc, ...) but we have also created a tool to help you generating combinatorial search spaces. 
##### For instance, lets say that we want to consider 3 solvents ($\bf{THF}$, $\bf{Toluene}$, $\bf{DMSO}$), 4 temperatures ($\bf{-10}$, $\bf{0}$, $\bf{10}$, $\bf{25}$) and 3 concentration levels ($\bf{0.1}$, $\bf{0.2}$, $\bf{1.0}$). We can introduce these in the EDBO+ scope generator in a dictionary form as follows:

In [3]:
reaction_components = {
    'Pd_precatalyst': ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10'],                
    'Ligand': ['L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 'L10'],                    
    'Pd_loading': [1,2,4,6,8,10],                                                       
    'Temperature': [30,40,50,60,70],                              
    'Selectfluor': [1,1.5,2],
}  

##### Now we need to pass the previous dictionary to the $\bf{generate\_reaction\_scope}$ tool in the EDBOplus class.

In [4]:
EDBOplus().generate_reaction_scope(
    components=reaction_components, 
    filename='Pd_fluorination_optimization.csv',
    check_overwrite=False
)

Generating a reaction scope...
The scope was generated and contains 9000 possible reactions!


,Pd_precatalyst,Ligand,Pd_loading,Temperature,Selectfluor
0,C1,L1,1,30,1.0
1,C1,L1,1,30,1.5
2,C1,L1,1,30,2.0
3,C1,L1,1,40,1.0
4,C1,L1,1,40,1.5
...,...,...,...,...,...
8995,C10,L10,10,60,1.5
8996,C10,L10,10,60,2.0
8997,C10,L10,10,70,1.0
8998,C10,L10,10,70,1.5


##### We can always load/read the previously generated reaction scope using any spreadsheet editor but in this case we will use Pandas for that:

In [7]:
import pandas as pd
df_scope = pd.read_csv('Pd_fluorination_optimization.csv')  # Load csv file.
n_combinations = len(df_scope)
print(f"Your reaction scope has {n_combinations} combinations.")
df_scope

Your reaction scope has 9000 combinations.


,Pd_precatalyst,Ligand,Pd_loading,Temperature,Selectfluor,yield1,yield2,yield3,priority
0,C2,L10,6,50,1.5,PENDING,PENDING,PENDING,1
1,C4,L3,6,50,1.5,PENDING,PENDING,PENDING,1
2,C5,L4,6,50,1.5,PENDING,PENDING,PENDING,1
3,C7,L2,6,50,1.5,PENDING,PENDING,PENDING,1
4,C5,L8,6,50,1.5,PENDING,PENDING,PENDING,1
...,...,...,...,...,...,...,...,...,...
8995,C4,L4,2,60,1.0,PENDING,PENDING,PENDING,0
8996,C4,L4,2,50,2.0,PENDING,PENDING,PENDING,0
8997,C4,L4,2,50,1.5,PENDING,PENDING,PENDING,0
8998,C4,L4,2,50,1.0,PENDING,PENDING,PENDING,0


##### Now we can check the number of combinations in the reaction scope:

In [5]:
n_combinations = len(df_scope)
print(f"Your reaction scope has {n_combinations} combinations.")

Your reaction scope has 9000 combinations.


##### Of course, this is a very small reaction scope  as it is meant to be a toy model to demonstrate how EDBO+ works.

## 2. First steps, initializing EDBO+ (in absence of training data).

##### We are going to execute EDBO+ to suggest some initial samples. 
##### Since we have not collected any experimental data (observations) yet, EDBO+ will suggest a set of initial experiments based on an feature space sampling method, in this case the CVT sampling method (see:http://kmh-lanl.hansonhub.com/uncertainty/meetings/gunz03vgr.pdf).
##### In this example the $\bf{objective}$ is to maximize the reaction $\bf{yield}$ and $\bf{enantioselectivity}$ but at the same time we want to minimize the amount one of a given $\bf{side~product}$ in this reaction. We also need to introduce the name of the csv file containing our reaction scope (in our case this was $\bf{my\_optimization.csv}$). Now we can execute the algorithm using the $\bf{run}$ command in EDBOplus:

In [5]:
EDBOplus().run(
    filename='Pd_fluorination_optimization.csv',  # Previously generated scope.
    objectives=['yield1', 'yield2', 'yield3'],  # Objectives to be optimized.
    objective_mode=['max', 'max', 'max'],  # Maximize yield and ee but minimize side_product.
    batch=5,  # Number of experiments in parallel that we want to perform in this round.
    columns_features='all', # features to be included in the model.
    init_sampling_method='cvt'  # initialization method (random, cvt, lhs)
)

There are no experimental observations yet. Random samples will be drawn.
The following columns are categorical and will be encoded using One-Hot-Encoding: ['Pd_precatalyst', 'Ligand']
Generated 5 initial samples using cvt sampling (seed = 0). Run finished!


,Pd_precatalyst,Ligand,Pd_loading,Temperature,Selectfluor,yield1,yield2,yield3,priority
1762,C2,L10,6,50,1.5,PENDING,PENDING,PENDING,1
2932,C4,L3,6,50,1.5,PENDING,PENDING,PENDING,1
3922,C5,L4,6,50,1.5,PENDING,PENDING,PENDING,1
5542,C7,L2,6,50,1.5,PENDING,PENDING,PENDING,1
4282,C5,L8,6,50,1.5,PENDING,PENDING,PENDING,1
...,...,...,...,...,...,...,...,...,...
2994,C4,L4,2,60,1.0,PENDING,PENDING,PENDING,0
2993,C4,L4,2,50,2.0,PENDING,PENDING,PENDING,0
2992,C4,L4,2,50,1.5,PENDING,PENDING,PENDING,0
2991,C4,L4,2,50,1.0,PENDING,PENDING,PENDING,0


##### EDBO+ has created a column for each objective and added $\bf{PENDING}$ values to all of them so you can track the experiments that you have been collecting during the optimization campaign.
##### We can also see that EDBO+ has created a new $\bf{priority}$ column. This column is used to distinguish between high and low priority samples. The top entries (with $\bf{priority=1}$) highlight the next suggested samples.

##### We can check now the first 5 experiments in the scope by reading the $\bf{my\_optimization.csv}$ file:

In [8]:
df_edbo = pd.read_csv('Pd_fluorination_optimization.csv')
df_edbo.head(5)

,Pd_precatalyst,Ligand,Pd_loading,Temperature,Selectfluor,yield1,yield2,yield3,priority
0,C2,L10,6,50,1.5,PENDING,PENDING,PENDING,1
1,C4,L3,6,50,1.5,PENDING,PENDING,PENDING,1
2,C5,L4,6,50,1.5,PENDING,PENDING,PENDING,1
3,C7,L2,6,50,1.5,PENDING,PENDING,PENDING,1
4,C5,L8,6,50,1.5,PENDING,PENDING,PENDING,1


## 3. Adding training data in EDBO+.

##### Note: We will use Python and Pandas to add new training data in this example. But you can always edit and add new data into the '.csv' file using any spreedsheet editor (such as Excel, Libreoffice Calc, ...) if that's more convinient for you.

##### Let's open again the $\bf{my\_optimization.csv}$ file we generated before:

In [8]:
df_edbo = pd.read_csv('Pd_fluorination_optimization.csv')
df_edbo.head(5)

,Pd_precatalyst,Ligand,Pd_loading,Temperature,Selectfluor,yield1,yield2,yield3,priority
0,C2,L10,6,50,1.5,PENDING,PENDING,PENDING,1
1,C4,L3,6,50,1.5,PENDING,PENDING,PENDING,1
2,C5,L4,6,50,1.5,PENDING,PENDING,PENDING,1
3,C7,L2,6,50,1.5,PENDING,PENDING,PENDING,1
4,C5,L8,6,50,1.5,PENDING,PENDING,PENDING,1


##### We can fill the first out entry in the previous dataframe with the "observed" values using Pandas:

In [9]:
df_edbo.loc[0, 'yield1'] = 20.5
df_edbo.loc[0, 'yield2'] = 40
df_edbo.loc[0, 'yield3'] = 37
df_edbo.loc[1, 'yield1'] = 50.3
df_edbo.loc[1, 'yield2'] = 10
df_edbo.loc[1, 'yield3'] = 0.2
df_edbo.loc[2, 'yield1'] = 32
df_edbo.loc[2, 'yield2'] = 10
df_edbo.loc[2, 'yield3'] = 22
df_edbo.loc[3, 'yield1'] = 20.5
df_edbo.loc[3, 'yield2'] = 40
df_edbo.loc[3, 'yield3'] = 0.1
df_edbo.loc[4, 'yield1'] = 66
df_edbo.loc[4, 'yield2'] = 10
df_edbo.loc[4, 'yield3'] = 11

##### We can check that we have filled out the first entry with our "observed data":

In [10]:
df_edbo.head(5)

,Pd_precatalyst,Ligand,Pd_loading,Temperature,Selectfluor,yield1,yield2,yield3,priority
0,C2,L10,6,50,1.5,20.5,40,37,1
1,C4,L3,6,50,1.5,50.3,10,0.2,1
2,C5,L4,6,50,1.5,32,10,22,1
3,C7,L2,6,50,1.5,20.5,40,0.1,1
4,C5,L8,6,50,1.5,66,10,11,1


##### Now we can save our dataset as $\bf{my\_optimization\_round0.csv}$:

In [11]:
df_edbo.to_csv('Pd_fluorination_optimization_round0.csv', index=False)

## 4. Running EDBO+ with training data.

##### First let's check our previous data (which include some $\bf{yield}$, $\bf{ee}$ and $\bf{side\_product}$ observations, which will be used to train the model):

In [12]:
df_edbo_round0 = pd.read_csv('Pd_fluorination_optimization_round0.csv')
df_edbo_round0.head(5)

,Pd_precatalyst,Ligand,Pd_loading,Temperature,Selectfluor,yield1,yield2,yield3,priority
0,C2,L10,6,50,1.5,20.5,40,37,1
1,C4,L3,6,50,1.5,50.3,10,0.2,1
2,C5,L4,6,50,1.5,32,10,22,1
3,C7,L2,6,50,1.5,20.5,40,0.1,1
4,C5,L8,6,50,1.5,66,10,11,1


##### Now that we have introduced some "observations" in our $\bf{my\_optimization\_round0.csv}$ file, we can execute EDBO+ to suggest samples using these "observations" as training data.

In [ ]:
EDBOplus().run(
    filename='Pd_fluorination_optimization_round0.csv',  # Previous scope (including observations).
    objectives=['yield1', 'yield2', 'yield3'],  # Objectives to be optimized.
    objective_mode=['max', 'max', 'max'],  # Maximize yield and ee but minimize side_product.
    batch=5,  # Number of experiments in parallel that we want to perform in this round.
    columns_features='all', # features to be included in the model.
    init_sampling_method='cvt'  # initialization method.
)

This run will optimize for the following objectives: ['yield1', 'yield2', 'yield3']
The following features will be used: ['Pd_precatalyst', 'Temperature', 'Pd_loading', 'Ligand', 'Selectfluor']
The following columns are categorical and will be encoded using One-Hot-Encoding: ['Pd_precatalyst', 'Ligand']
Generating surrogate model...
Model generated!

🔍 Computing SHAP feature importance...
  [1/3] Analyzing yield1...


  0%|          | 0/10 [00:00<?, ?it/s]

C:\Users\D.Dalmau\OneDrive - unizar.es\Desktop\PROGRAMS\EDBO+\edboplus\edbo\plus\optimizer_botorch.py:654: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(shap_vals, X_train, feature_names=feature_names,
C:\Users\D.Dalmau\OneDrive - unizar.es\Desktop\PROGRAMS\EDBO+\edboplus\edbo\plus\optimizer_botorch.py:663: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(shap_vals, X_train, feature_names=feature_names, show=False)


    ✓ Saved plots and values
  [2/3] Analyzing yield2...


  0%|          | 0/10 [00:00<?, ?it/s]

...

    ✓ Saved plots and values
  [3/3] Analyzing yield3...


  0%|          | 0/10 [00:00<?, ?it/s]

C:\Users\D.Dalmau\OneDrive - unizar.es\Desktop\PROGRAMS\EDBO+\edboplus\edbo\plus\optimizer_botorch.py:654: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(shap_vals, X_train, feature_names=feature_names,
C:\Users\D.Dalmau\OneDrive - unizar.es\Desktop\PROGRAMS\EDBO+\edboplus\edbo\plus\optimizer_botorch.py:663: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(shap_vals, X_train, feature_names=feature_names, show=False)


    ✓ Saved plots and values

✅ SHAP analysis complete! Results in: ./shap_analysis

Optimizing acqusition function...
Acquisition function optimized.
Predictions and expected improvement obtained.
Run finished!


,Pd_precatalyst,Ligand,Pd_loading,Temperature,Selectfluor,yield1,yield2,yield3,priority
184,C9,L8,1,30,1.0,PENDING,PENDING,PENDING,1.0
1972,C7,L8,1,70,1.0,PENDING,PENDING,PENDING,1.0
1984,C7,L8,1,30,1.0,PENDING,PENDING,PENDING,1.0
4309,C5,L2,1,70,1.0,PENDING,PENDING,PENDING,1.0
7914,C10,L2,1,30,1.0,PENDING,PENDING,PENDING,1.0
...,...,...,...,...,...,...,...,...,...
8999,C2,L10,6,50,1.5,20.5,40,37,-1.0
1,C2,L10,6,40,1.5,34,2,23,-1.0
2,C2,L10,4,70,1.5,56,8,7,-1.0
3,C2,L10,2,70,2.0,90,5,2,-1.0


##### Again the samples suggested by EDBO+ have $\bf{priority = +1}$. In addition, we asign $\bf{priority = -1}$ to the experiments that we have already run (these are at the bottom of the dataset).

## Extra: Accessing the model predictions.

##### Each time that EDBO+ is executed with training data it will generate a .csv file with the predictions for the entire scope (including the 'untested' samples).
##### In the previous example, by running EDBO+ with training data, we generated two files: $\bf{my\_optimization\_round0.csv}$ and a second file with the predictions $\bf{pred\_my\_optimization\_round0.csv}$.
##### Let's have a look to the predictions file using Pandas:

In [ ]:
df_predictions_round0 = pd.read_csv('pred_Pd_fluorination_optimization.csv')
df_predictions_round0.style.background_gradient(subset=['priority'], cmap='plasma')

##### In the previous dataset we can access the model predictions ($\bf{predicted\_mean}$ and $\bf{predicted\_variance}$ columns) but also to the expected improvement values (in the $\bf{expected\_improvement}$ columns) for each objective ($\bf{yield}$, $\bf{ee}$ and $\bf{side\_product}$.).